--- Day 7: Recursive Circus ---
Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
...then you would be able to recreate the structure of the towers that looks like this:

                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         qoyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

In [346]:
import re

class DiscProg():
    complete_set = {}
    
    @classmethod
    def clear_set(cls):
        cls.complete_set = {}
    
    def __init__(self):
        self.name = None
        self.weight = None
        self.children = []
        self.parent = None
        self._total = None
        self._norm = None
        
    def __hash__(self):
        return self.name
    
    def is_root(self):
        if self.parent:
            return False
        else:
            return True
        
    def find_root(self):
        if not self.parent:
            return self
        else:
            return self.parent.find_root()
        
    @property
    def total(self):
#         if self._total:
#             return self._total
        weight = self.weight
        for child in self.children:
            weight += child.total
        self._total = weight
        return weight
    
    @property
    def norm(self):
        if self._norm:
            return self._norm
        
        for i in range(len(self.children)-1):
            if self.children[i].total == self.children[i+1].total:
                self._norm = self.children[i].total
        return self._norm
        
    def unbalanced(self):                
        for child in self.children:
            if child.total != self.norm:
                print("unbalanced child: {} ({}) -> {}".format(child.name, child.norm, [(x.name, x.total) for x in child.children]))
                if child.children:
                    return child.unbalanced()
                else:
                    return child
                
    
    def __repr__(self):
        parent = self.parent
        if parent:
            root = parent.name
        else:
            root = 'root'
            
        kid_out = ["{} ({})".format(x.name, x.weight) for x in self.children]
            
        return "{} ({}) -> {}".format(self.name, root, kid_out)
    
    @classmethod
    def by_name(cls, name):
        if name in cls.complete_set:
            return cls.complete_set[name]
        else:
            ret = DiscProg()
            ret.name = name
            cls.complete_set[name] = ret
            return ret
        
    @classmethod
    def from_input(cls, record):
        kids = []
        out = None
        
        if '->' in record:
            me, kids = record.split('->')
            kids = [cls.by_name(x.strip()) for x in kids.split(',')]
        else:
            me = record
            
        mtc = re.match('([a-z]+?)\s\((\d+)\)', me)
        
        if me and mtc:
            name = mtc.group(1).strip()
            out = DiscProg.by_name(name)
            out.weight = int(mtc.group(2))
            if kids:
                for k in kids:
                    k.parent = out
                out.children = kids
            cls.complete_set[out.name] = out
        return out
    
    @classmethod
    def parse_input(cls, inpt):
        recs = inpt.split('\n')
        return [cls.from_input(x) for x in recs]

In [347]:
test = """
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
"""

In [354]:
discs = DiscProg.parse_input(full)

In [357]:
root = DiscProg.by_name('xcpuy').find_root()
root.unbalanced()

In [358]:
DiscProg.by_name('drjmjug').weight

420

Norm is 48284
unbalanced child: smaygo (eugwuhl) -> ['hmgrlpj (66)', 'fbnbt (5262)', 'nfdvsc (4824)']
Norm is 14564
unbalanced child: fbnbt (smaygo) -> ['lcmek (741)', 'aflzz (1359)', 'rqgccu (760)', 'vxfpzku (797)', 'yvixrl (7)', 'ojkhavf (1465)']
Norm is 1549


In [272]:
fbnbt = DiscProg.by_name('drjmjug')

In [265]:
fbnbt.parent

smaygo (eugwuhl) -> ['hmgrlpj (66)', 'fbnbt (5262)', 'nfdvsc (4824)']

In [324]:
full = """
xcpuy (20)
prxlb (35)
yibbu (160)
pgvds (134) -> manpd, vkikri
kzvbkr (24)
bwhjal (68)
vozygf (134) -> fzyftv, yiozhnf, hxscfiv
ynzpsk (17)
uwmbh (34)
juqrwwm (98) -> bhtoyru, ulacz, enkufi
ctjnwq (22) -> tasqwda, qfybq
ibjoxjo (175) -> ubhsnqw, icjmhii, lmvfuql, wrnmwy
puxlsjo (26) -> gyveijp, nikvfs
aaeeqh (68) -> nmyem, fserz
sfxng (33)
uvhyl (23)
gtkkx (91)
ovkpba (352)
dkjfjwe (146) -> ehnlfa, sxzwplf
fjklelc (83) -> pjkrhl, yshith
gzmejde (318) -> eghfd, oxcou
suhso (32)
cnvir (249) -> zghut, fjrwy
rgzch (92)
cqnlk (229) -> lpvqp, pdowqq
insxh (186)
qutgb (83)
byhatd (1716) -> zeenwpx, tqyte, nrgwyzb
lfzthdo (85)
aftndu (11) -> xckzcq, cymvz, oblhfhz
nxkpw (34)
btyotwv (139) -> asbiye, nmvqwi, swwtm, ctytwcc
zbwysvr (1226) -> ckdwk, bjzlgik, lbajhe, lnoihzx
jqwhun (39)
ohyrn (68)
gwvhjfe (13)
fczix (226) -> yvdilfo, cprsra
acrly (75)
lfoiu (13)
fxptqn (242) -> srqrs, mxgqn
emcjby (96)
ztpysan (63)
bfioj (56)
ovtoke (56)
pgkiqa (75)
muesucc (50)
hlyodf (27)
drvxvk (102) -> bmzlp, elpfsya
cnhsgdv (264)
ioexd (52) -> shfyok, xcwlym
juvwqt (63)
gaspe (54)
qkqyelb (253) -> ehravmh, qzqtysl
hekihe (18)
jljwoo (171) -> uqajm, dgyjldr
yornm (149) -> nniqh, gqcmcc
tqqldx (74)
bxqdudx (195) -> dyacjh, nsecr, unfpyg
fqbovid (93)
yzexxy (68)
xfwgll (8)
omytneg (1366) -> hmvin, eppzprf, zrioi, sdlcqtb
dgjqciq (24)
wxtrq (340) -> eolll, wawkzh
ctdghk (57)
oninxl (46)
kxjnq (92)
klccwcn (22)
ktyyedq (61)
tqyte (225) -> xpzymh, wdyxx
ajmtxv (267) -> sjmci, yhdjwd
uraugcn (73)
mztwrvf (34) -> kmvcjir, cgwnor
kozlia (37)
pbxguu (28)
ktooxbh (74)
qjcsn (65)
pvvbn (8732) -> adyni, dvltfhm, luerna
avroyn (58)
mdkfx (54)
fbdrf (80)
evqfv (62)
enkufi (62) -> vurdsd, srpcy, dihezs, xzcix
lxngrgu (56)
xyhoa (33)
jcqgj (46) -> nxqswd, lznju, kohici
ysnwjl (29)
szcqd (260) -> gpzqcqk, tnngnq
wlcjt (298)
rjhws (97)
srqrs (46)
xnfstk (61)
ixukpw (77)
cujtqk (74)
nfvliuy (97)
iondfqm (45)
vzlvim (10)
wijfihq (156) -> zauue, mgqziu, egrlfh
iyosp (88) -> gcpipz, scsxe
bdohoaa (18440) -> lxjbzu, ppsnn, hatfl
lrxyqgr (106) -> phppbj, tbure, sspayll
ffejzo (70) -> lcourm, rdfab
wawkzh (16)
mnvlfcf (132) -> icmmx, hsnsncv, bsmryo
kbpnl (10)
pnqrgfk (1707) -> msekejl, aaroa, tfregkr
gjmkox (57)
fufzi (191) -> evhgkft, kzroqbi
uoyzsdf (20)
otxroc (56)
gtmcr (578) -> svsgzi, pnmkl, owsvc
orbwii (77)
mkqtej (81)
nmpxqi (205) -> prspd, uaguc
ltseugc (37)
kfyaw (76)
dgyjldr (62)
rsvudoc (1348) -> zwvtdhw, gpvlztd, njvsnz
mxgqn (46)
yjbjy (98)
ftrzswp (84)
nzakpnw (50) -> gjjgkga, bvgct, pzbgf
iufypzm (58)
utizol (25)
bwkwgf (206) -> huhxf, txzlem, rlfbix, itzdby
ydehaq (67)
ovuqg (22)
scsxe (49)
jfuzgpw (34)
hdtxgb (22)
snluaf (45)
dhpviei (25)
kfxcot (182) -> hxqik, lkstwl, vqumdju
zgcfeda (8)
fixsej (22)
cyrqkl (86)
risguql (97)
fvyiwup (165) -> hdtxgb, ovuqg, klccwcn, fixsej
mzlub (83)
lhcinxf (58)
xjszzfe (92)
uqqclia (4793) -> gtmcr, ohitm, ashxpav
mbcpufw (85)
urxzo (145) -> ntqdxv, nxnjrzj, fcbeagx, aqxdf
qpycn (544) -> yornm, iiygchg, vpvcum, plbmu, bxqdudx, twfzzyt
basiy (133) -> qifofxk, qutgb
dvpay (74)
kridw (175) -> edmfm, jrigbij
tmbkt (94)
zibll (188)
mhdauqb (89)
bilxm (96)
qmgot (69)
hxscfiv (93)
soixton (66)
lbajhe (180) -> fausgp, zkeep
edbemy (286)
mwkct (11)
iwkqae (35)
ilbcji (205) -> ctxwkkt, utizol
wusxwq (1078) -> npzlka, wxsvnj, kirvox, pqguzw, tecducv
xqssuna (10)
cslrgbh (64)
aubapo (11)
twfzzyt (175) -> rdomi, pcwawh
hhrvet (301) -> mskxqki, jwtzgjf
iarbj (93)
uhukwcq (151) -> pnncrgq, yuepo
rnkvgzf (78)
usevrmp (96)
zdcdmle (97)
nwhunwf (88) -> ygidtwv, lrxzg, ifluvxj, soixton
qfybq (95)
ahrfh (2040) -> yyfrmb, actlu
zgvpy (200) -> nxkpw, hhpysjd
ahnkcy (83)
woqvn (54)
ktfbjoo (90) -> jgsydq, yjbjy
usfzaf (25)
iaini (32)
jarvx (188) -> jjlsy, nrywfm
fuyldc (70)
cdovdt (410)
eqxce (23) -> qtatze, wwoye, qkseme, yrtlizo, kridw, birvaha
vkgqysh (47) -> cpmxvjn, aebhvbk, epubm
xppkv (249) -> uasrsi, qfqkz, rkznog
kbmrrew (38)
tkhgo (59)
cebdo (17)
holuib (302) -> usfzaf, oustcqz
obqblrl (83)
crxji (97)
lqsgphu (91)
gldorrf (198) -> byymvpo, ulkpoko
oujtz (69) -> cyrqkl, vjcexcx, wuztvge
ulkpoko (68)
pwdziu (253) -> nwhma, igbhn, ckyoc, kfvur
zxqmj (20)
iartkgt (91)
hcorylb (97)
nkfpci (87)
qdmqsrh (80)
fkfyeej (299)
jkfce (7)
rwkuzs (29)
fzqdsv (38) -> yptyp, wohpoh
aqxdf (16)
bvdnvep (18)
nxnjrzj (16)
csdrrx (323) -> rwkuzs, jbdtgk
tubcr (46)
ebootd (83)
lkuelr (62)
prspd (25)
eooiuq (54) -> qiqjmmi, gperhh, qanpeab, bwkwgf, jgmyc
zhpmkq (89)
cexrf (72)
gzilup (14)
rvzkv (70) -> wgsgyv, neafgd, oseqshg
aqdjk (80)
owsvc (433)
vkybxfk (56)
dwnys (55)
iiktsrr (17)
tfzsqj (89) -> pqknhsq, fhbjovx, jbaxm, pvnfa, eqxce, hbmzilw, hyxxm
pnmkl (325) -> rhyglf, meybf
bwntinc (13)
dbrxe (69)
nrsrwb (45)
vinbva (278) -> jpjezb, zfswj
ojkhavf (1465) -> siujrb, igxzggb
kuusoe (396) -> gxcwynf, mxvhj, wijfihq
vjmzqbb (89)
jggzx (20)
liida (77)
wlxwq (70)
izjeb (105) -> lduim, hgegrz, veegvvc, pvzeznc
akxtfh (57)
avflv (46)
qkmqmg (40)
eflcxi (21)
kexdopt (21)
wuvctps (43)
tpjuj (24) -> fnvptes, bjtlzzh, tyajj, izjeb, kfxcot, wvhdyg
jvsmwee (88)
gqlnaho (84) -> tlfaxub, xjszzfe
hazyoke (40)
fapkonf (3295) -> hmlvv, xppkv, ezwitpp
cjinsf (80) -> wwyuko, renqhyi
bvoqpkb (18)
wdyxx (47)
tlnffr (58)
dlodc (70) -> zhpmkq, xbiuhd
lpjbjju (38)
ifluvxj (66)
rxerq (168) -> uoyzsdf, seyiftq, ydfmxw, ezgvb
nplbgrk (59)
cyjcaht (13)
hxeia (271) -> fzvllyz, hidmpw
wgjrzd (93) -> touatc, grbpi, lclxb, grqtsex
tkoxc (1050) -> pmafsug, jbaztvm, rsvudoc, kixlvf, ogjeuhl, tthmac, qpycn
yuzwdix (43)
vtewtb (85) -> thablkl, hyywt, baljt, insvmt
dihezs (91)
jorbfl (181) -> uvhyl, bqjypo
rkznog (9) -> civhlqc, xytulz
bxdbijl (63)
nkywqaj (40)
oppmeig (60) -> yqhss, suupi
dkjsfxs (59)
igxzggb (42)
drjmjug (428) -> goiwg, sbogacc, jdvxawc, wlczi, vinbva
lclxb (80)
bjdlb (50) -> yonojr, fwubo
ckakkja (38) -> fqbovid, siulcz, bapvqa, pwajo
igbhn (40)
cwwpev (17)
maqzwn (37)
rhtamt (85)
ezwitpp (123) -> aftndu, urxzo, qchxbon
edaslb (190) -> robfhmz, qbqjz
crlkolg (36)
izjppbx (1108) -> qzvrq, cgnewz, zbwysvr, xkghfhn, ugujla, nwfykuw
aebhvbk (92)
ohjax (56)
ozvtq (254) -> zjchuge, zxqmj, xcpuy
fuvpi (83)
gdneruv (2040) -> srgfuk, btyotwv, butfc
maubg (85) -> cnxyhg, mpgge, uyfzha
ckgotfs (94) -> bilxm, emcjby
vtqpilo (75)
tfjadw (41)
paqbnl (39)
mudup (87)
zgxczmq (98)
egrlfh (20)
icgwqn (84)
lxouzag (61)
qbfsj (60)
fbnbt (5262) -> lcmek, aflzz, rqgccu, vxfpzku, yvixrl, ojkhavf
nvkqru (139) -> sdqzod, raucx
gnrij (65)
kwtghnf (25)
azxoyif (1368) -> dkohgp, irgkfsa, plysamb
yrmqu (14)
xcwlym (14)
yvixrl (7) -> stguc, dfywfie, ialmhy, pkpgu, nzakpnw, ompeys
wwyuko (52)
cujdv (18)
lxuiwb (201) -> ubnap, qbdisnk, urqoabs, oepwc
touatc (80)
ubllrlw (8)
idqaxgi (78)
yxotxz (42)
pnncrgq (51)
qnpzxal (75)
bhfeafe (91) -> pfgsvs, ruzgpq
nsdver (97)
uqajm (62)
cxapyp (261) -> aodkqlr, lmxvhwl, ralmgdq
uzqnl (220) -> xyhoa, sfxng
yeimwk (56)
oblhfhz (66)
ppsnn (7698) -> hhrvet, ajmtxv, wubrts, nyjgkav, lfvih, kyrookv
oarmggv (14)
zkqxpbq (97)
lnreval (98)
zvbnb (28)
kpxrk (45)
gfhuv (40)
bqjypo (23)
gghqilu (186) -> sxxnze, paioxf, gjmbm, kwtghnf
lvsqvdi (13)
lvuah (66)
mkuernu (60)
zvqyr (36)
sspayll (39)
twzglew (14)
rnvvg (456) -> dkjfjwe, pbeez, cjinsf
hlxlrjq (48) -> dzrsha, stkvqy
lhypfq (51)
mdwiktj (15)
mytnwmk (58)
bvgct (69)
yedwosl (18)
jvneo (51)
brdpdm (34)
apkfhsb (36) -> mwknwxh, mqrvu
tgvzdov (40) -> pubgkhb, angyt
rppgdnf (11) -> tbkxpdv, vugudp, etyqg, zlbfpnu
ehefb (2158) -> akxtfh, uxqlce
owsoywa (99)
swtks (14)
qiqjmmi (10) -> hgznhq, zknngyu, lkhvhui, tcmexa, edaslb, gzmejde
nebitb (19)
hkmaxza (221)
hpaeg (54) -> lvqkp, xjbmdc, solooh
avjfqex (74)
lmntml (74)
sfpveq (38)
svsgzi (45) -> nsdver, xexswl, oidmfs, gwlrdbk
kbpklq (173) -> oajdkzc, lxouzag
vlbkhb (93)
tjjkgb (69)
tbfce (5) -> vozygf, mjmskc, rludm, wgjrzd, pwdziu
nwhma (40)
gabbp (70)
beyphy (221) -> cwfgzrp, ojnrf
kqjfnb (91) -> tlnffr, avroyn, mytnwmk, zamoi
lfpwycg (209) -> qiniyf, vdtdpz
zyqkcmb (130) -> ynbruov, dqvygcd, gzilup, oarmggv
uaguc (25)
eatoyd (91)
taizyha (45)
uembuig (39)
cpmxvjn (92)
bfaak (89)
bkrctx (84) -> usevrmp, bkrbfap, weaier
eppzprf (120) -> zvbnb, pbxguu
nrgwyzb (263) -> eunfszw, aijmghs
qucnu (225)
igblzk (97)
hgegrz (53)
uaknxb (99)
lnoihzx (43) -> qkhudy, zdseqzi, ydehaq
dejsmq (41)
plbmu (107) -> rcaajf, qdmqsrh
naseaj (61)
jdvxawc (26) -> kfyaw, kglxbse, ukkmfh, jyoevqm
sierb (113) -> asihmc, iaini, wepwf
kglxbse (76)
sigya (98)
manpd (82)
rodegh (88)
lhauud (220) -> zlaxh, eqtmes
sspng (199) -> dkcrll, mnvfi
tntib (246) -> ldypun, aubapo
vjstu (84)
rjkumo (97)
qqncuk (77)
suyzrmf (155) -> ybfyes, cndqzdm
kyrookv (181) -> mdxcp, zdcdmle
viubix (10)
epblwo (314) -> ysnwjl, mryuhsl
itluk (97)
ovuesou (85)
jqeabja (81) -> luulwx, jjanr
zefoig (27)
blbzqf (209) -> vnlcfxn, yqeuhw
hkfgksw (22)
pbeez (64) -> qbfsj, xprrajl
tbure (39)
hobnerj (89)
bonoob (78)
qylxl (277) -> ttiaks, pijlcc
jgsydq (98)
hafgurf (95)
nrvqpd (58)
mqrvu (44)
srgfuk (109) -> hewzwp, ienee, uwmbh
ldypun (11)
pkivl (217) -> kdwzsz, dejsmq
ywacbmq (87)
slsrzl (60) -> fqskwj, lhypfq, jvneo, mwbpro
icqkc (13)
edvqwvm (60)
ivrnod (42)
xpjksm (46)
pfgsvs (84)
xkghfhn (1524) -> hdlfpj, deqcnp, iomxv
euqxp (77)
hdkvbzv (50) -> aqubpi, twogxyg, vcrsnrf, xezse
cixqrb (62)
ofxag (36) -> flyxei, jpwtw
yypfse (46)
hgznhq (302) -> tkqdly, kexdopt
xgdolgm (59)
pzbgf (69)
uymwt (24)
zwiidh (85)
ubnap (21)
tkdsfz (17)
yfwki (55) -> jqgjdku, jbsxc, lcsqza
yonojr (53)
geavik (8)
luerna (10160) -> fpekhb, hyolftw, rnvvg
yxccn (86) -> dknly, xtbdj
povsm (58)
eolll (16)
syfccrr (92)
lbgwc (24)
rkevnnd (81)
wqtku (66)
uspjc (84)
iiygchg (151) -> cwehc, cwkdjn, pppsd, ydnsnh
mzdxg (51) -> yqdsxvc, bxdbijl
bvfee (35)
kuhybd (73)
flwhrzk (31)
djkaku (44)
oaefi (9) -> yigwd, nldvtr
xqsufx (61)
kyfcsou (68)
ienee (34)
ojnrf (35)
jwtzgjf (37)
jajbpsk (39)
wxsvnj (205) -> rabdvok, vrbjw
mwknwxh (44)
pijlcc (9)
ulacz (412) -> ghzvdod, jkfce
axfvokp (35)
ugujla (78) -> fybpzfy, uiyxk, selfpoj, fdurqvg, kdsscve, jarvx
uvjevu (45)
evmkzjr (8)
mdfjh (58)
mqafr (88)
rdbbda (73)
jywye (68)
nvlpgon (88)
qqqvdsu (160) -> cyjcaht, bwntinc, hhrdosf, lvsqvdi
bcwbp (43)
jpjezb (26)
ujhkl (13)
tjwodb (63)
etyqg (71)
xrppd (57)
ppstt (61)
pwmqpk (62)
xpzymh (47)
stkvqy (78)
mljlpfo (90) -> gnrij, lalblzp
byymvpo (68)
zauue (20)
cqdyyoo (401)
wwoye (211) -> ojhbiv, cixqrb
pjkrhl (56)
rpiefl (120) -> cjnkge, xqmhigw, ugeucjn, brdpdm
pvzeznc (53)
yigwd (84)
bbiczh (13)
vdgvmd (18)
qchxbon (109) -> muesucc, vvukaca
btpbnr (31)
wuztvge (86)
iegjd (275) -> geavik, zgcfeda
oydwkrv (69)
dhmhbw (50)
krhkthl (52)
umkwllw (7)
amgrlkr (75)
sfztpf (84)
glfoc (15)
xprrajl (60)
ralmgdq (91) -> bmcsya, lfzthdo
jqvjr (47) -> adzvlgk, fdvgk, lnttud, aspoy
ebeitz (1424) -> utdhy, hfzyzt, cidfent, ctjnwq, fjkjmhd
ihqfqm (365) -> zixgz, glfoc, ieabomm
dzrsha (78)
cwfgzrp (35)
mskxqki (37)
ywhxoi (94)
lmavxq (75) -> idqaxgi, bonoob
mpmmos (56)
wklmecc (66) -> amgrlkr, vtqpilo, nnnfykw
oajdkzc (61)
izkhkai (45)
mcxti (62)
lztme (46)
mxvhj (216)
ljqmwwg (41)
dhrun (37)
tkgkl (40)
ifvkdba (52) -> ibuilzl, yalugkr, vxbio, pgkiqa
hgzsxur (10) -> iwczlkt, qmgot, tjjkgb, umjdo
jjanr (57)
cjnab (72)
yffnz (1004) -> smvodeb, ztruf, apkfhsb
rmbzvq (19)
wvaketk (46)
qjhdxdr (74)
thablkl (18)
chrzza (66) -> pnfwlas, igbqenq, hmytpw, nwhunwf
vhusoux (1834) -> slsrzl, cnhsgdv, bzvqqp
qcdsvv (82)
pzfdzb (87)
dxlttzs (70)
pvnfa (503) -> rwjlopt, aqcwqy, ilbcji, suyzrmf, nmpxqi, ibjoxjo
utdhy (100) -> owkjav, vkybxfk
aqcwqy (101) -> httgiox, orbwii
qiveb (197) -> gpqcq, evpav, btpbnr
hbmzilw (1829) -> wnaykx, ncrotbi, jywye
qklhqlf (46)
shyxe (93)
gounu (145) -> azvltq, jrquxee
kbehv (62) -> dbbsyeo, jotnlgm, igqzqiy, uhukwcq, zhallzk, fvyiwup, wgkxy
rtwwdrh (417) -> wbflq, sierb, cawvue
qlesgoz (185) -> pwlpeu, rchqa
rwjlopt (11) -> xqsufx, bjwgltq, naseaj, zjioec
jjlsy (83)
mryuhsl (29)
ehravmh (37)
xjpuoil (31)
jbwoozn (111) -> otxroc, bfioj
wubrts (330) -> jjsbd, mdwiktj, qnuusbq
fxnbfjk (8)
luufi (53)
cjnkge (34)
xzjbmpu (323) -> zefoig, zljgpq
ijajh (55)
pmebb (42)
jerwsi (80)
lmxvhwl (175) -> azbjh, wuvctps
pqguzw (181) -> kqtye, xpjksm, oninxl
qqkctf (121) -> zouklk, busosd
rchqa (98)
ojhbiv (62)
zjioec (61)
bjzlgik (154) -> iondfqm, izkhkai
eoxubh (92)
ftzhz (18)
fwjdwhg (62)
txkog (46)
rabdvok (57)
dfykpx (59)
lncdocs (70)
qyqid (69)
quybmbk (179) -> pmfuf, ubrevf
qkhudy (67)
grqtsex (80)
dijeym (61)
ancmoj (74)
lpvqp (17)
angyt (75)
tecducv (151) -> ftrzswp, sfztpf
podkl (78)
ieqgany (17)
nnnfykw (75)
bjtlzzh (133) -> syfccrr, rgzch
xoedja (1123) -> chrzza, cghtkzr, gzrfjk
pvdsg (74)
mqsth (258) -> bcwbp, idnvd, amhthnw, rfmgglc
wnaykx (68)
mdxcp (97)
jbsxc (68)
rcaajf (80)
reeuuic (66)
zhallzk (173) -> qkmqmg, tkgkl
kzroqbi (19)
lygtf (24) -> gqaefmb, xgowghi, nbxceyt, tboimc, rptgp, cdovdt
fmntbo (204)
mgwbaj (21)
zlxxdjc (54)
lalblzp (65)
rdfab (43)
gxprou (58) -> wyucu, grwcz, kbyijkc
qkseme (44) -> axnwb, zkqxpbq, crxji
vjcexcx (86)
nhmrdn (99)
gqaefmb (234) -> wdqjl, kbkqm
aenvjl (34) -> rknadpx, qreeg, wevbhta
koyzdbq (20)
idnvd (43)
qewavw (61) -> gkyph, byfbs, lhcinxf
htqjli (38)
jtqgtf (55)
evpav (31)
axnwb (97)
gzrfjk (781) -> sieovol, lmavxq, qqkctf
iauzkqz (81) -> axjcbuj, meqlt
vtdyp (180) -> twxtbol, cfihl, vucwhs
iomxv (226)
sihgu (99)
azysfe (45)
iwczlkt (69)
zeenwpx (22) -> nhmrdn, sihgu, uaknxb
jbaxm (293) -> qiveb, fczix, xrhjl, gavdkw, ptcccmw, puawqd
hirayas (57)
zwvtdhw (84) -> tujzoy, gtkkx
plbbbno (25)
sdlcqtb (76) -> gfepc, hxkatz
qbqjz (77)
tkqdly (21)
seyiftq (20)
lznju (96)
uamaay (43) -> ucjeppl, lfmzadj
tbkxpdv (71)
puoogd (95)
zkkok (1239) -> qtxfdex, fufzi, uamaay
bfkdbeh (162) -> blmsqb, vzeuayg, kbpnl, viubix
tthmac (1206) -> maubg, qewavw, jpeehl, pvzjsbf
hhldn (61) -> cujtqk, pvdsg, lmntml
zouklk (55)
nmvqwi (18)
cgnewz (179) -> gounu, icflka, yusue, zbjdebj, mevdoy, gvgnev, etfbf
sletcp (31)
adzvlgk (63)
fnoaz (96)
aaroa (121)
iksafw (41)
zfswj (26)
zwekns (66)
pkhvge (95)
fpkhun (185) -> jtqgtf, fwfad
bapvqa (93)
tyajj (135) -> jvhjtv, pxupwhx
lfmzadj (93)
alhkjxm (67) -> qqncuk, lywsab
lkhvhui (224) -> uufmp, qumacc
pqmyl (11)
qrdei (86)
fnvptes (297) -> cjnwipw, blwxqfp
ftuues (124) -> puoogd, gyhfl
azbjh (43)
epubm (92)
vusafsv (6) -> ivxnetm, syjew, csdrrx, ngonq, azfkwzp, awysiko, qlesgoz
jrigbij (80)
khbpnx (117) -> kpegrqm, mudup
qxyemg (7)
gkyph (58)
vbusaxq (68)
kxgxji (372)
lvihecc (33)
mjpcrh (13)
kpegrqm (87)
ohapkri (19)
phppbj (39)
hewzwp (34)
siujrb (42)
pnfwlas (232) -> mkuernu, edvqwvm
qrxso (46) -> azxoyif, kbehv, zuhkp
tpezujs (41)
zghut (5)
elpfsya (83)
vsvyj (242)
irkfc (14) -> rypyfha, cgagfib
nmyem (90)
njkxh (71)
ynxjgv (72)
ohvhpc (36)
tlfaxub (92)
iazmq (17)
yhdjwd (54)
vqumdju (45)
olylo (188)
dmkji (1094) -> hdkvbzv, cfyuogb, puxlsjo
dukup (72)
djpvps (31)
bokni (97)
jludr (7)
sdqzod (44)
uryqiso (183) -> rmbzvq, lvpdwy
eugwuhl (18) -> avpklqy, tytbgx, bdohoaa, smaygo, pvvbn, hgizeb, tchfafn
wgkxy (141) -> ryhlz, ohjax
ttiaks (9)
mpgge (50)
hdlfpj (46) -> bddii, konmy, mdwqmyg
mxoxo (117) -> icgwqn, vjstu
kbpxjg (225)
xthyeh (38)
dyxtot (51)
tcmexa (322) -> xhueuj, pqmyl
kbjazf (12) -> ljqmwwg, iksafw, tpezujs, tfjadw
sdnwo (75)
fwubo (53)
bfnkr (132) -> wlodpu, wavmnm, hcpim
eymlu (58)
fybpzfy (210) -> pqmsc, wufcpvr
tnngnq (27)
nsecr (24)
xhueuj (11)
hsnsncv (22)
oseqshg (76)
aflzz (1359) -> pkhvge, uyuyq
urqoabs (21)
aiwqhey (53)
hxqik (45)
zvsiqwe (63)
etfnmle (76)
eqreeik (27) -> vrezl, lvuah, ffrjb, ykuzfu
joxzy (253) -> fxnbfjk, czvdrpq, ubllrlw, hjsfv
zamoi (58)
okcck (61)
bwyjnu (60) -> pzuxx, mdfjh
ydfmxw (20)
upeezd (564) -> ozjzmh, fzbbm, fjhht, wrzwgxf, nvkqru, jorbfl
hxlsxdj (19)
hsvveid (10)
xbiuhd (89)
cfyuogb (13) -> hlyodf, tvtgaan, qsewkq
kndobxz (75)
wlfngqq (484) -> pgvds, rvzkv, azycte, igoifwl, zfgqhf, wlcjt
ctoruu (72) -> ynxjgv, cexrf
ashxpav (59) -> quybmbk, sydqf, qnblxrh, ysfevlj, xgpmq, stgickl
njibgv (688) -> ahmknyy, vruiy, wlfngqq, juvhkoc, ehefb, qfuoayy
elcvqr (19)
hmvin (176)
lkstwl (45)
neafgd (76)
dbbsyeo (79) -> jldgbkx, nkfpci
ohitm (1133) -> aaeeqh, dlodc, rxerq
ivbrype (72)
jgmyc (1366) -> oaefi, mzdxg, muzkuai, hblnwza
kixlvf (1906) -> eledia, ioexd, mamllly
ngonq (243) -> oydwkrv, rnibhd
ubqbhos (87)
eevyny (77)
vwrhg (72)
fwfad (55)
smaygo (4616) -> hmgrlpj, fbnbt, nfdvsc
arbrp (48) -> vlysbd, njkxh
ggzzt (41) -> fkilb, iartkgt
cxflah (122) -> oxyltk, clrfkgl
pkpgu (95) -> nnzhve, wcvxg
httgiox (77)
lnttud (63)
nvpgfzh (68)
ootknbf (45)
eledia (6) -> maqzwn, iplfnd
srpcy (91)
jpeehl (157) -> paqbnl, jqwhun
clwbp (45)
xubhat (70)
umwer (17)
rnibhd (69)
zfgqhf (160) -> dbrxe, fyubj
zfngau (94)
ctytwcc (18)
igbqenq (212) -> fuyldc, dxlttzs
quhkp (88)
wrzwgxf (77) -> kgdtem, kndobxz
twngkp (59)
gperhh (1195) -> sspng, pjtsef, blbzqf
nyjgkav (191) -> jwvcm, kxjnq
tijgc (94)
bsoesu (47) -> mhdauqb, bfaak
xzcfko (21) -> bdslm, bywohrp, tkhgo
wwpxh (51) -> luufi, aiwqhey
gwlrdbk (97)
bbueq (52)
kidbaiy (38)
gqcmcc (59)
uasrsi (95) -> cujdv, bvdnvep, vzldrq, vdgvmd
lxjbzu (12) -> nsqfu, nxkpbs, ebeitz, lygtf
syhjziw (24)
jqjmgyv (85)
dvhcixe (17)
sqcwjaa (22)
robfhmz (77)
qfbagb (42)
pubgkhb (75)
wdqjl (88)
qanpeab (9) -> qylxl, kymwev, fpkhun, jljwoo, kbpklq, rppgdnf, qjjuk
vsyolk (299)
vgysvve (75)
wbflq (145) -> wdwfjh, suhso
izchcj (83)
kfvur (40)
fvagct (176)
tksiwle (78) -> xjwvvik, pwmqpk
iquxck (6442) -> zubbg, tpjuj, zkkok, upeezd, nafjtip
hojpgpo (58) -> shyxe, xkzzcqt, ymqqpug, eqmkgj
qzvrq (1178) -> gxprou, rpiefl, xzndm, tchaqbm
ogjeuhl (2016) -> argdqd, euezshv
ieeecih (98)
rhyglf (54)
txyqy (178) -> bbiczh, lfoiu
fjkjmhd (192) -> ntjmdri, vzlvim
qzqtysl (37)
yqdsxvc (63)
wufcpvr (72)
ewalq (70)
xeeuxiz (33) -> rkevnnd, mkqtej
ubhsnqw (20)
vkikri (82)
pooon (22) -> bkrctx, tybjmqm, wxtrq, hgynpc, epblwo, hnswwt, kxgxji
xsepbvk (77)
xexswl (97)
igrahel (146) -> uzbqa, ckakkja, ihqfqm
lvqkp (66) -> jvsmwee, quhkp, cufau
zubbg (1077) -> hhldn, kvyrc, fhsdpg
gvgnev (289)
icjmhii (20)
dbsbuqp (97) -> ahnkcy, ebootd
ygsjv (137) -> fzqita, llale
tjnuf (180) -> mwkct, qclnd
pdowqq (17)
zkeep (32)
dknly (78)
bmzlp (83)
cghtkzr (418) -> holuib, ovkpba, ifvkdba
cymvz (66)
tqzmvz (80)
npmkxsb (84)
hyolftw (39) -> vkgqysh, kqjfnb, hxeia
fjrwy (5)
dfywfie (63) -> nfvliuy, risguql
aspoy (63)
ydnsnh (29)
jyoevqm (76)
jacplh (133) -> ahrynjm, aenrksb
yiozhnf (93)
yqhss (76)
kmujst (284) -> iwsppaw, axjqq, mljlpfo
trnpdnb (78)
hhrdosf (13)
zrvff (210) -> odhwzy, sfpveq
odirpnl (20)
rknadpx (63)
brnqhc (70) -> neclo, ppstt, ktyyedq, xnfstk
sctzxf (88)
bddii (60)
vrndjn (19)
swwtm (18)
cqxav (72)
rzaocr (98)
juavhi (20) -> krhkthl, bbueq, gdmuqn
vugudp (71)
zoifh (63)
qwwmx (71)
fdvgk (63)
xzcix (91)
cufau (88)
qbdisnk (21)
ozjzmh (81) -> lvlfju, uraugcn
mwbpro (51)
zrioi (156) -> hsvveid, xqssuna
xqmhigw (34)
fufjkl (57)
kvyrc (133) -> sdnwo, acrly
bhtoyru (186) -> icxhhje, ighndd, fbdrf
gfluo (55)
ighndd (80)
blmsqb (10)
gjjgkga (69)
bsmryo (22)
owkjav (56)
kymwev (215) -> nkywqaj, hazyoke
tzpxj (134) -> ujhkl, gwvhjfe
ymqqpug (93)
rfmgglc (43)
evhgkft (19)
nikvfs (34)
argdqd (65)
sxzwplf (19)
pydtt (45)
ctxwkkt (25)
axjcbuj (35)
pustj (742) -> juavhi, bwyjnu, fvagct, kbjazf, oyomma
dapds (32)
hphzmhs (17)
wavmnm (65)
fcdww (40)
eunfszw (28)
qifofxk (83)
npzlka (99) -> lijmpm, dwnys, ijajh, gfluo
nhtxs (252) -> dvhcixe, uwopuvj
qvjwgkp (160) -> tjwodb, juvwqt
fzbbm (137) -> gheqw, nrsrwb
lduim (53)
zowqczm (96)
fqyogt (45)
yxavos (89)
gyhfl (95)
bjwgltq (61)
vljijeh (150) -> xfwgll, evmkzjr
cawvue (81) -> cslrgbh, vkpnxs
ijlmcnx (78)
mjmskc (399) -> slolo, jludr
htpboal (69)
mjseecw (68)
wjmbivd (63) -> rodegh, dwhxnif, mqafr
eqtmes (47)
muzkuai (77) -> dhmhbw, gqwfut
kohici (96)
cwgux (18)
iwsppaw (150) -> axfvokp, qagul
pqmsc (72)
qnuusbq (15)
nofdxpt (491) -> iauzkqz, cayxz, ygsjv
wgsgyv (76)
tchfafn (31649) -> qrxso, xoedja, fapkonf
pwlpeu (98)
gpqcq (31)
cnxyhg (50)
ryhlz (56)
kmvcjir (89)
slolo (7)
fqdccsa (91)
xtbdj (78)
nniqh (59)
jwvcm (92)
myerj (204)
oepwc (21)
awclg (49)
igoifwl (176) -> dijeym, okcck
qcsjl (33)
asihmc (32)
fpekhb (344) -> lqjnx, xhqxyc, swxncwt, vljijeh
xytulz (79)
vlysbd (71)
busosd (55)
kbkqm (88)
gpvlztd (106) -> tqzmvz, aqdjk
cgagfib (95)
jbdtgk (29)
rpjlb (68)
xazpni (92)
meybf (54)
wlczi (246) -> yxotxz, xronh
birvaha (275) -> koyzdbq, odirpnl, jggzx
oavnz (22)
blwaz (247) -> nebitb, vrndjn
yalugkr (75)
yusue (91) -> owsoywa, pobecj
hfzyzt (124) -> mhvhtg, djkaku
jvjezrv (31)
uyuyq (95)
scghvn (98)
pndayy (1340) -> ftzhz, yedwosl
smvodeb (124)
hgynpc (267) -> bvfee, prxlb, iwkqae
owjjja (1495) -> lfpwycg, vqntk, xzjbmpu
wvhdyg (29) -> dlbsqa, fnoaz, zowqczm
jotnlgm (79) -> povsm, nrvqpd, iufypzm
fzyftv (93)
bdslm (59)
stguc (133) -> evqfv, lkuelr
fagyf (589) -> jjxcsxg, khbpnx, wklmecc, ajxmlfp, iegjd, eqreeik, beyphy
stgickl (229) -> kozlia, znuin
lviohn (71)
actlu (15)
gxcwynf (80) -> kyfcsou, jelcp
azvltq (72)
kgdtem (75)
juaquyw (164) -> uembuig, jajbpsk
eyfvei (103) -> zgxczmq, lnreval
hcpim (65)
rludm (351) -> sletcp, djpvps
gjmbm (25)
ffrjb (66)
kyokkl (89)
zjchuge (20)
uxqlce (57)
gtnqtd (59)
uufmp (60)
jqgjdku (68)
rlyui (73)
fyubj (69)
twxtbol (49)
xdehk (63)
pobecj (99)
iplfnd (37)
guyju (86)
jbaztvm (53) -> jqvjr, eyfvei, fkfyeej, paqngfj, basiy, pkivl, vsyolk
mamllly (66) -> qxyemg, umkwllw
pxupwhx (91)
nxkpbs (1707) -> cnvir, yfwki, bhfeafe
xezse (11)
qreeg (63)
puawqd (256) -> cwwpev, eihkuqx
yrtlizo (291) -> letwfzz, oavnz
hyywt (18)
dfqvxnn (73)
letwfzz (22)
alcioc (314)
msekejl (77) -> utvdgr, hkfgksw
ybfyes (50)
bzvqqp (86) -> vjmzqbb, gzakvk
iavld (19)
syjew (204) -> tgtrjko, xgdolgm, jngbnk
pwzvu (66)
dnlao (51)
oidmfs (97)
ckyoc (40)
etfbf (239) -> oitpd, nhxyn
lvpdwy (19)
xzuedm (212) -> dyxtot, dnlao
hdsth (32) -> qrdei, guyju
ygidtwv (66)
rypyfha (95)
gdmuqn (52)
nbxceyt (320) -> taizyha, pydtt
pzuxx (58)
vpvcum (91) -> nvlpgon, sctzxf
lkokq (77)
nrevor (596) -> xeeuxiz, jqeabja, fjklelc, dkgsxvj
konmy (60)
nigdlq (846) -> hdsth, hlxlrjq, txyqy, myerj, fmntbo, irkfc
djsexcr (22)
aodkqlr (185) -> ymxpav, kbmrrew
wubxbc (43) -> htqjli, lpjbjju, hbncy
xlyzym (57)
xgpmq (126) -> dkjsfxs, dfykpx, gtnqtd
kmjqzw (63) -> avjfqex, tqqldx, ancmoj
nmteeca (80)
twogxyg (11)
cwkdjn (29)
tasqwda (95)
cayxz (33) -> nplbgrk, twngkp
ovzcdi (42)
fmfkcp (71)
ompeys (207) -> dhpviei, plbbbno
ucjeppl (93)
mrshnke (18)
ytbwqwb (51)
lrxzg (66)
luasrvp (1476) -> mnvlfcf, ofxag, xzcfko
fhbjovx (1553) -> tmmwda, yibbu, tzpxj
wrquv (50) -> zlxxdjc, mdkfx, woqvn
odhwzy (38)
onmod (99) -> flwhrzk, jvjezrv, umopgq, xjpuoil
bhwxgp (74)
hmlvv (75) -> bsoesu, qucnu, kbpxjg
eghfd (13)
zknngyu (12) -> izchcj, bcxnm, mzlub, itcgsx
vruiy (711) -> aenvjl, onmod, ggzzt, daiyp, lrxyqgr, uvdpata, jbwoozn
xmett (76)
wrnmwy (20)
civhlqc (79)
tchaqbm (121) -> snluaf, uvjevu, kpxrk
skouwt (671) -> qvjwgkp, uzqnl, gghqilu, kpmyxk, ckgotfs, edbemy, nhtxs
uxddlkq (32)
lcourm (43)
ibuilzl (75)
pwajo (93)
nafjtip (1182) -> cxflah, insxh, iyosp, zyqkcmb
fserz (90)
yshith (56)
vvukaca (50)
pcwawh (46)
mevdoy (289)
hmgrlpj (66) -> pnqrgfk, luasrvp, ahrfh, tbfce, drjmjug, nigdlq, omytneg
xkzzcqt (93)
xronh (42)
lywsab (77)
icflka (157) -> pwzvu, wqtku
iqegs (345) -> yrmqu, swtks, twzglew, fmcjr
zlaxh (47)
itooipv (68)
ieiha (97)
sxrzzuy (13) -> hcorylb, itluk, ybjbz, igblzk
tgtrjko (59)
qclnd (11)
qfqkz (15) -> xmett, etfnmle
weaier (96)
yznahfs (102) -> wubxbc, vtewtb, jacplh, wwpxh, zhdwu, digpoav
xrhjl (216) -> dhrun, ltseugc
jngbnk (59)
tujzoy (91)
aijmghs (28)
ieabomm (15)
siulcz (93)
yqeuhw (42)
hxkatz (50)
gpzqcqk (27)
nnzhve (81)
evwtu (72)
gqwfut (50)
huhxf (299) -> uspjc, npmkxsb
ukkmfh (76)
rptgp (296) -> xthyeh, kidbaiy, rjumsra
wlodpu (65)
utvdgr (22)
qjjuk (109) -> iarbj, vlbkhb
mnvfi (47)
gfepc (50)
mkomwy (86) -> ktfbjoo, zrvff, hgzsxur
fzvllyz (26)
whrdhq (2578) -> uymwt, dgjqciq
nwfykuw (2136) -> jonbur, izuxb
ooiwbe (49)
nhxyn (25)
hyxxm (743) -> hojpgpo, hragl, mqsth
sxxnze (25)
swxncwt (34) -> zwekns, reeuuic
ruuerqk (43)
zhdwu (17) -> ewalq, gabbp
hidmpw (26)
kpmyxk (247) -> icqkc, mjpcrh, uvcsl
jjsbd (15)
csvni (19)
gavdkw (38) -> ztpysan, zvsiqwe, zoifh, xdehk
darwcoj (2168) -> yffnz, igrahel, dmkji, juqrwwm, pndayy, nrevor
selfpoj (46) -> xsepbvk, liida, dyaayff, ywhmyp
lvlfju (73)
azycte (298)
gheqw (45)
tnntrax (34)
nfdvsc (4824) -> kenpyrt, ytqrut, pustj, hbymky, tpvvbr, lcfsu
qsewkq (27)
pxncq (98)
dyaayff (77)
umopgq (31)
yuepo (51)
plysamb (11) -> yuwoa, vwrhg
hbzkf (51)
qiniyf (84)
oitpd (25)
kqdus (52) -> bwhjal, vbusaxq
tytbgx (6588) -> ynseh, uqqclia, darwcoj, eooiuq
unuvi (52) -> pvxof, qcdsvv
eihkuqx (17)
sbogacc (264) -> qcsjl, wttimqr
ajxmlfp (225) -> lvihecc, yanat
rqryd (132) -> pmebb, qfbagb
raucx (44)
xckzcq (66)
uiyxk (204) -> vgysvve, qnpzxal
kirvox (159) -> jerwsi, nmteeca
wepwf (32)
zvqupa (98)
uzbqa (376) -> umwer, iazmq
lcsqza (68)
qhlslym (147) -> gaxkntb, eymlu
fqskwj (51)
hragl (244) -> kfpxhb, mcxti, fwjdwhg
hjsfv (8)
ncrotbi (68)
ymxpav (38)
ntqdxv (16)
ynbruov (14)
sieovol (231)
pqknhsq (1031) -> gldorrf, jcqgj, fxptqn
yviutlw (31) -> gjmkox, hirayas, xlyzym
paqngfj (161) -> qyqid, htpboal
wohpoh (75)
hatfl (6172) -> sqygj, nofdxpt, mkomwy, kmujst
cgwnor (89)
jelcp (68)
azfkwzp (339) -> eflcxi, mgwbaj
nadwsc (113) -> ohvhpc, zvqyr, crlkolg
fjhht (193) -> ieqgany, hphzmhs
edmfm (80)
cjnwipw (10)
hvgum (2381) -> dfqvxnn, kuhybd, rdbbda, rlyui
gyveijp (34)
bcxnm (83)
vzldrq (18)
cfihl (49)
nxqswd (96)
xhnwej (85)
uvcsl (13)
kenpyrt (282) -> zgvpy, tntib, drvxvk, htojdr, gqlnaho
fzqita (7)
vznfkp (97) -> zfngau, tijgc
rlfbix (79) -> bokni, rjhws, ieiha, rjkumo
ckswfrb (48)
hbncy (38)
paioxf (25)
zuhkp (1365) -> bjdlb, ihimta, ffejzo
aqubpi (11)
vkpnxs (64)
itcgsx (83)
xjbmdc (142) -> tmbkt, ywhxoi
vucwhs (49)
vuldc (70)
lcmek (741) -> tksiwle, bfkdbeh, tjnuf, yviutlw
ysfevlj (205) -> awclg, ooiwbe
grwcz (66)
yuwoa (72)
cprsra (32)
tvtgaan (27)
pjtsef (59) -> podkl, rnkvgzf, bqwgm
cwehc (29)
wcvxg (81)
ahmknyy (637) -> qkqyelb, vtdyp, wjmbivd, bfnkr, oujtz
yieehh (259) -> lviohn, qwwmx
hgizeb (5324) -> tfzsqj, njibgv, izjppbx
jvhjtv (91)
ksuhtas (14) -> qvykht, wahfxls
fkilb (91)
emigqf (46)
xgowghi (256) -> lkokq, eevyny
vqntk (21) -> hobnerj, yxavos, kyokkl, mzrze
vrbjw (57)
wttimqr (33)
pvzjsbf (143) -> wvaketk, qklhqlf
solooh (188) -> ffcwobn, fmfkcp
qmygary (95)
ztruf (44) -> gfhuv, fcdww
qtxfdex (191) -> iavld, elcvqr
mzrze (89)
hhpysjd (34)
bqwgm (78)
qfuoayy (74) -> xzuedm, ftuues, alcioc, lhauud, ozvtq, szcqd, brnqhc
yvdilfo (32)
grbpi (80)
khjqt (87)
avpklqy (68) -> werlnym, tkoxc, iquxck
jjxcsxg (18) -> eatoyd, lqsgphu, fqdccsa
kdsscve (310) -> sqcwjaa, djsexcr
veegvvc (53)
umjdo (69)
bkrbfap (96)
baljt (18)
qorpdxg (72)
wahfxls (88)
unfpyg (24)
zdelwv (57)
fhsdpg (143) -> lncdocs, xubhat
ahrynjm (12)
pvxof (82)
ynseh (5204) -> hpaeg, yznahfs, cxapyp, rtwwdrh, kuusoe
bmcsya (85)
thzvetu (65)
uyfzha (50)
ghzvdod (7)
rjumsra (38)
qvykht (88)
awysiko (93) -> qorpdxg, ccdpnr, evwtu, cjnab
daiyp (133) -> clwbp, azysfe
bywohrp (59)
pppsd (29)
qagul (35)
axjqq (106) -> xrppd, fufjkl
uvdpata (131) -> emigqf, tubcr
vrezl (66)
fausgp (32)
wjyekj (56)
ckdwk (104) -> wlxwq, vuldc
dyacjh (24)
yanat (33)
tboimc (18) -> sigya, ieeecih, scghvn, zvqupa
pmfuf (62)
oxyltk (32)
gzakvk (89)
shfyok (14)
znuin (37)
tpvvbr (974) -> rqryd, ctoruu, unuvi
vzeuayg (10)
hmytpw (178) -> ywacbmq, ubqbhos
vxfpzku (797) -> zibll, kqdus, fzqdsv, olylo
ovhos (46)
amhthnw (43)
cndqzdm (50)
adyni (11474) -> kmjqzw, mxoxo, joxzy, lxuiwb, vznfkp, blwaz
eqmkgj (93)
lijmpm (55)
sqygj (848) -> jiwpf, ckswfrb
gcpipz (49)
czvdrpq (8)
suupi (76)
zljgpq (27)
werlnym (34) -> gdneruv, byhatd, wusxwq, vusafsv, skouwt, hvgum
hnswwt (32) -> mbcpufw, xhnwej, ovuesou, jqjmgyv
txzlem (467)
mdwqmyg (60)
vdtdpz (84)
ntjmdri (10)
zlbfpnu (71)
digpoav (61) -> dapds, jwutq, uxddlkq
kfpxhb (62)
nldvtr (84)
ybjbz (97)
ptcccmw (116) -> khjqt, pzfdzb
rdomi (46)
goiwg (162) -> lxngrgu, ovtoke, mpmmos
ialmhy (206) -> iiktsrr, acgef, tkdsfz
dkohgp (119) -> bvoqpkb, cwgux
nhbff (51)
nrywfm (83)
qtatze (299) -> mrshnke, hekihe
rnzpmw (92)
mgqziu (20)
dlbsqa (96)
jrquxee (72)
meqlt (35)
hbymky (774) -> qqqvdsu, mztwrvf, wrquv, oppmeig
sjmci (54)
oyomma (125) -> ynzpsk, jygrpp, cebdo
ubrevf (62)
ezgvb (20)
lqjnx (166)
llale (7)
tybjmqm (372)
byfbs (58)
fmcjr (14)
gaxkntb (58)
jonbur (33)
asbiye (18)
vcrsnrf (11)
zapjqj (54)
deqcnp (142) -> ivrnod, ovzcdi
jpwtw (81)
renqhyi (52)
tfregkr (53) -> tnntrax, jfuzgpw
clrfkgl (32)
cidfent (104) -> zapjqj, gaspe
vxbio (75)
ykuzfu (66)
itzdby (331) -> itooipv, nvpgfzh
acgef (17)
ehnlfa (19)
hblnwza (7) -> rhtamt, zwiidh
igqzqiy (167) -> yuzwdix, ruuerqk
sydqf (31) -> rpjlb, mjseecw, ohyrn, yzexxy
xjwvvik (62)
xhqxyc (74) -> avflv, ovhos
dvltfhm (54) -> pooon, fagyf, owjjja, vhusoux, whrdhq
aenrksb (12)
rqgccu (760) -> qhlslym, cqnlk, dbsbuqp
euezshv (65)
juvhkoc (1546) -> yxccn, juaquyw, vsvyj
jygrpp (17)
zixgz (15)
irgkfsa (83) -> syhjziw, kzvbkr, lbgwc
dqvygcd (14)
fcbeagx (16)
ugeucjn (34)
dwhxnif (88)
insvmt (18)
zbjdebj (73) -> dukup, cqxav, ivbrype
kbyijkc (66)
qumacc (60)
dkgsxvj (5) -> hafgurf, qmygary
zdseqzi (67)
nsqfu (1724) -> tgvzdov, ksuhtas, arbrp, nbpkmt
njvsnz (128) -> lztme, txkog, yypfse
fdurqvg (198) -> trnpdnb, ijlmcnx
oxcou (13)
neclo (61)
wyucu (66)
lfvih (99) -> rnzpmw, eoxubh, xazpni
xzndm (199) -> hxlsxdj, csvni, ohapkri
jwutq (32)
jldgbkx (87)
luulwx (57)
ivxnetm (267) -> zdelwv, ctdghk
butfc (58) -> nhbff, hbzkf, ytbwqwb
pmafsug (1041) -> hkmaxza, uryqiso, alhkjxm, pnjgt, nadwsc
dkcrll (47)
ccdpnr (72)
mhvhtg (44)
icmmx (22)
pnjgt (55) -> obqblrl, fuvpi
ytqrut (1468) -> euqxp, ixukpw
ruzgpq (84)
blwxqfp (10)
yyfrmb (15)
ffcwobn (71)
uwopuvj (17)
lmvfuql (20)
htojdr (72) -> rzaocr, pxncq
qnblxrh (7) -> qjhdxdr, bhwxgp, dvpay, ktooxbh
vnlcfxn (42)
wdwfjh (32)
tmmwda (70) -> fqyogt, ootknbf
jiwpf (48)
ywhmyp (77)
yptyp (75)
izuxb (33)
oustcqz (25)
wevbhta (63)
kdwzsz (41)
flyxei (81)
lcfsu (18) -> cqdyyoo, sxrzzuy, iqegs, yieehh
ihimta (44) -> wjyekj, yeimwk
vurdsd (91)
icxhhje (80)
kqtye (46)
nbpkmt (60) -> thzvetu, qjcsn
"""